## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,73.26,42,75,16.11,broken clouds
1,1,Beringovskiy,RU,63.0500,179.3167,-16.42,86,2,10.00,clear sky
2,2,Tasiilaq,GL,65.6145,-37.6368,14.16,54,100,8.70,overcast clouds
3,3,Kodiak,US,57.7900,-152.4072,20.71,42,0,14.97,clear sky
4,4,Albany,US,42.6001,-73.9662,40.68,66,100,4.25,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,73.26,42,75,16.11,broken clouds
5,5,Pizarro,CO,4.9533,-77.3660,74.84,90,90,2.68,overcast clouds
6,6,Tuatapere,NZ,-46.1333,167.6833,74.64,50,20,3.06,few clouds
7,7,Dicabisagan,PH,17.0818,122.4157,80.04,70,56,5.79,broken clouds
9,9,Atuona,PF,-9.8000,-139.0333,78.55,76,10,17.49,clear sky
12,12,Jardim,BR,-21.4803,-56.1381,71.22,47,5,4.14,clear sky
16,16,New Norfolk,AU,-42.7826,147.0587,72.91,31,10,9.31,clear sky
17,17,Cayenne,GF,4.9333,-52.3333,75.24,92,75,5.01,light rain
20,20,Touros,BR,-5.1989,-35.4608,81.50,71,78,8.81,broken clouds
21,21,Butaritari,KI,3.0707,172.7902,81.79,77,100,15.26,light rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                217
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,73.26,broken clouds,-42.8794,147.3294,
5,Pizarro,CO,74.84,overcast clouds,4.9533,-77.3660,
6,Tuatapere,NZ,74.64,few clouds,-46.1333,167.6833,
7,Dicabisagan,PH,80.04,broken clouds,17.0818,122.4157,
9,Atuona,PF,78.55,clear sky,-9.8000,-139.0333,
12,Jardim,BR,71.22,clear sky,-21.4803,-56.1381,
16,New Norfolk,AU,72.91,clear sky,-42.7826,147.0587,
17,Cayenne,GF,75.24,light rain,4.9333,-52.3333,
20,Touros,BR,81.50,broken clouds,-5.1989,-35.4608,
21,Butaritari,KI,81.79,light rain,3.0707,172.7902,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        
hotel_df.head()        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,73.26,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
5,Pizarro,CO,74.84,overcast clouds,4.9533,-77.3660,Hotel Limonar
6,Tuatapere,NZ,74.64,few clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
7,Dicabisagan,PH,80.04,broken clouds,17.0818,122.4157,Nancy Montanes Bernardo
9,Atuona,PF,78.55,clear sky,-9.8000,-139.0333,Villa Enata


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   217
Country                217
Max Temp               217
Current Description    217
Lat                    217
Lng                    217
Hotel Name             217
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Desc</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))